<a href="https://colab.research.google.com/github/ffabi/VisualAnalysis/blob/main/M%C3%A9r%C3%A9si_solar_HF_AEP0TG_OZFKFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Specifikáció

Házi feladat téma (rövid megnevezés) *:

Napelem teljesítményének vizsgálata az időjárás függvényében

Házi feladat leírás (miről szól a feladat) *:

A napelemes rendszerek teljesítménye számtalan tényezőtől függ: panelek száma, alkalmazott technológia és annak hatásfoka, dőlésszög, árnyékolódás, az egyenlítőtől mért távolság, aktuális időjárási jellemzők, stb. Projektünkben egy bizonyos napelemrendszer teljesítményét fogjuk vizsgálni az aktuális időjárás függvényében. A teljesítmény- és időjárásadatokból akár két részletes kimutatást is lehetne készíteni, a napelem esetében a hosszabbtávú trendekről, az időjárás esetében pedig például annak előrejelzéséről. A két adatforrás feldolgozása együtt fog majd teljes képet alkotni, viszont ez a komplexitás esetében több mint kétszeres szorzót jelent, ez az oka a csapatmunkának.

Házi feladatban megválaszolandó kérdések *:

A napelemes rendszerek teljesítménye nem csak az évszaktól, illetve a felhőréteg vastagságától, hanem az aktuális hőmérséklettől is függ, ezek azok az összefüggések, amiket valószínűleg könnyedén ki lehet majd mutatni. 
További érdekességeket is lehet hogy fel tudunk majd tárni, például a napelemek tisztítási időpontjait, illetve a hosszútávú teljesítménycsökkenést, stb.

Házi feladatban felhasznált adatforrások (megnevezés, URL, kb. adatmennyiség) *:

Daily Power Production of Solar Panels:

https://www.kaggle.com/fvcoppen/solarpanelspower

Napi bontásban tartalmazza egy antwerpeni napelemes rendszer termelését, illetve a háztartás fogyasztását és a fűtésre használt gáz mennyiségét.
Az adathalmaz 8 éves időtartamot fed le, így a hosszútávú trendek is megfigyelhetőek lesznek.

Weather dataset in Antwerp, Belgium

https://www.kaggle.com/ramima/weather-dataset-in-antwerp-belgium

Órás (néhol félórás) bontásban tartalmazza Antwerpen időjárási adatait, mintpéldául: hőmérséklet, szélerősség, légnyomás, páratartalom, látótávolság, szöveges időjárás.
Az adatbázis 7 éves időtartamban szolgáltat időjárásinformációkat, amik nagyrész lefedik a napelemadatbázis adatait.

A két adatbázis kombinálásánál fellépő egyik probléma a különböző időbeliség.

Tervezett megjelenítés jellege (dashboard, riport, webalkalmazás, stb.) *:

Dashboard a releváns interaktív grafikonokkal.

Link a statisztikai summary dokumentumra *:

Kaggle Data Explorer:

https://www.kaggle.com/fvcoppen/solarpanelspower

https://www.kaggle.com/ramima/weather-dataset-in-antwerp-belgium




## Solar data description
Source: https://www.kaggle.com/fvcoppen/solarpanelspower

Context
In oktober 2011 we installed solar pannels (or Photovoltaic Modules) on our roof. The total power of the modules is 5kWp.
It may seem strange but we have the habbit of making daily recordings of our electricity usage and so it was obvious to make record of the powerproduction of the solar pannels. I am trying to predict the date of the next 1000kWh produced.

Content
The file PVElecGas2.csv is ';' separated and has 4 columns: Date, Cumulative solar power, used kWh electricity, used Gas m²
csv file has two columns: first the date and second the cumulative power in kWh.

Acknowledgements
We wouldn't be here without the help of others. I have until now worked with the excellent information and sample python programs from Jason Brownlee : https://machinelearningmastery.com/

Inspiration
I am still working to improve the performance. I hope some one can get better results in predicting the future power production for the next year.

## CSV description:

This is my cumulative daily PV panels solar power production. Also columns with daily electric power and gas consumption . The measurements are taken in the morning between 7h and 8h , so it is correlated to the weather of the day before.

The columns in each file are:

- date: date in the DD/MM/YYYY format

- Cumulative_solar_power: The daily value of the power meter on the DC to AC converter. This is the current cumulative solar power.

- grid_usage: This is the daily power consumption of the house. This value can be negative in the case that the solar power exceeds

- Gas_mxm: This is the daily gas consumption of the house. Gas is only used for heating.

https://www.kaggle.com/ramima/creating-and-merging-weather-data-with-solar-power

## Weather data description

Source: https://www.kaggle.com/ramima/weather-dataset-in-antwerp-belgium

Context
Days ago, applying my skills in web-scraping, I brought this data. After I found that there is no dataset that gathers both weather and solar power energy, so you can analyze and predict solar production depending on weather status; Based on that, I decided to create such a dataset.

Content
Once I found a good dataset describing solar panel production here on kaggle. It remains to get the weather data of the same area where that panels locate (Antwerp, Belgium). So I've scraped it from Time And Date website.
This Dataset has the weather status (wind, humidity, air pressure, and temperature) in Antwerp, Belgium from 2012 to 2019. The same period of time for the Daily Power Production of Solar Panels dataset.

Acknowledgements
We wouldn't be here without the help of others. A big thanks to Frank for his permission to use his Daily Power Production of Solar Panels dataset here. And thanks for the team of Timeanddate website for their free weather site.

Inspiration
By adding Frank's dataset to your notebook and merge it with this weather dataset, after cleaning both, you will be able to chase answers for questions like:
1- What are the weather factors that most affect the amount of renewable energy produced by solar panels?
2- Should we turn the solar panels off in some months totally, in case they produce no energy?
3- Does the air pressure influence energy production even if it's a sunny day?

It's worth noting that the orientation of the solar panels - says Frank - is WSW. So most of the energy comes in the afternoon with a peak between 15:00 and 19:00 in the summertime.

## CSV description:

Weather status in Antwerp city in the period between 2012-2019



The columns in each file are:

- id

- clock: Time in hours

- temp: Temperature in Celsius degree

- weather:Sky status

- wind: Wind speed in km/h

- humidity: Humidity percent

- barometer: Atmospheric pressure in millibar

- visibility: Distance of visibility in Km

- year

- month

- day




http://www.solarelectricityhandbook.com/solar-angle-calculator.html

TODO: 2015-16 tél hideg volt? Ki kéne hozni

# Load solar data and preprocess it

In [33]:
#import libraries
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose

# Download data
os.environ['KAGGLE_USERNAME'] = "ffabian"
os.environ['KAGGLE_KEY'] = "a06083fe469934fbd0ec143ea1681c92"
!kaggle datasets download fvcoppen/solarpanelspower -q
!unzip -q -u solarpanelspower.zip

In [59]:
# load data from csv file
solar = pd.read_csv('PV_Elec_Gas3.csv', ',')

solar = solar.rename(columns = {"Unnamed: 0" : "date"})
solar = solar.rename(columns = {"kWh electricity/day" : "grid_usage"})
solar = solar.rename(columns = {"Gas/day" : "Gas_mxm"})

# create separate year, month, day values
solar["date"] = pd.DatetimeIndex(pd.to_datetime(solar["date"], format="%d/%m/%Y"))
solar['year'] = pd.to_numeric(solar['date'].dt.strftime('%Y'))
solar['month'] = pd.to_numeric(solar['date'].dt.strftime('%m'))
solar['day'] = pd.to_numeric(solar['date'].dt.strftime('%d'))

# calculate daily generated energy
solar = solar.set_index("date")
solar2 = solar.shift(periods=1, freq='D', axis=0)
solar['Cumulative_solar_power_shift'] = solar2.loc[:,'Cumulative_solar_power']
solar['daily_generated_energy'] = solar['Cumulative_solar_power'].values - solar['Cumulative_solar_power_shift'].values

# solar = solar[(solar["year"] >= 2012)]
solar = solar[solar["daily_generated_energy"].notna()] 

solar = solar.drop(['Cumulative_solar_power'], axis=1)
solar = solar.drop(['Cumulative_solar_power_shift'], axis=1)
solar = solar.reset_index()

# create is_hot column featuring low gas usages
solar["is_hot"] = solar["Gas_mxm"] <= 3
solar["is_hot"] = solar["is_hot"].astype(int)

solar["consumption"] = solar["grid_usage"] + solar["daily_generated_energy"]

solar = solar.sort_values(by="date").reset_index(drop=True)

The preprocessed solar data looks as following:

In [63]:
solar

,date,grid_usage,Gas_mxm,year,month,day,daily_generated_energy,is_hot,consumption
0,2011-10-27,7.4,9.2,2011,10,27,10.1,0,17.5
1,2011-10-28,5.8,8.0,2011,10,28,10.0,0,15.8
2,2011-10-29,4.9,6.6,2011,10,29,9.4,0,14.3
3,2011-10-30,11.7,5.3,2011,10,30,4.6,0,16.3
4,2011-10-31,11.0,5.7,2011,10,31,3.8,0,14.8
...,...,...,...,...,...,...,...,...,...
3298,2020-11-06,16.0,11.0,2020,11,6,8.0,0,24.0
3299,2020-11-07,13.0,13.0,2020,11,7,8.0,0,21.0
3300,2020-11-08,12.0,11.0,2020,11,8,8.0,0,20.0
3301,2020-11-09,14.0,10.0,2020,11,9,5.0,0,19.0


The main descriptor values for each column are the following:

In [64]:
solar.describe()

,grid_usage,Gas_mxm,year,month,day,daily_generated_energy,is_hot,consumption
count,3303.000000,3303.000000,3303.000000,3303.000000,3303.000000,3303.000000,3303.000000,3303.000000
mean,4.581865,8.343506,2015.838329,6.540721,15.721768,11.041144,0.319104,15.623009
std,9.856519,6.335050,2.620775,3.453081,8.817438,8.292842,0.466200,3.448840
min,-24.000000,0.000000,2011.000000,1.000000,1.000000,0.000000,0.000000,-2.000000
25%,-3.000000,2.000000,2014.000000,4.000000,8.000000,4.000000,0.000000,14.000000
50%,6.000000,8.000000,2016.000000,7.000000,16.000000,9.000000,0.000000,16.000000
75%,13.000000,13.000000,2018.000000,10.000000,23.000000,17.000000,1.000000,18.000000
max,34.000000,29.000000,2020.000000,12.000000,31.000000,34.000000,1.000000,47.200000


In [37]:
solar.nunique()

date                      3304
grid_usage                 124
Gas_mxm                     91
year                        10
month                       12
day                         31
daily_generated_energy     455
is_hot                       2
consumption                395
dtype: int64

In [38]:
############################# TEMPLATE ##########################
#@title
#Double-click the empty area on the right to hide the code
# title = "template"
# fig = px.bar(solar, 
#              x="year", y="daily_generated_energy", 
#              template="plotly_white", title=title)
# fig.show()
# fig.write_html(title.replace(" ", "_") + ".html")


In [39]:
#@title
helper = solar.set_index("date").groupby(pd.Grouper(freq='M')).mean()
date_axis = helper.index

result = seasonal_decompose(helper["consumption"], model='additive')

fig = make_subplots(rows=4, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1)

fig.add_trace(go.Scatter(x=date_axis, y=result.observed),
              row=1, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.trend),
              row=2, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.seasonal),
              row=3, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.resid),
              row=4, col=1)

fig.update_xaxes(title_text="Time", row=4, col=1)
fig.update_yaxes(title_text="Observed", row=1, col=1)
fig.update_yaxes(title_text="Trend", row=2, col=1)
fig.update_yaxes(title_text="Seasonal", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

fig.update_layout(
    height=600, 
    width=800,
    title_text="Seasonal decompose monthly consumption", 
    template="plotly_white",
    showlegend=False
)
fig.show()

In [40]:
#@title
helper = solar.set_index("date").groupby(pd.Grouper(freq='M')).mean()
date_axis = helper.index

result = seasonal_decompose(helper["daily_generated_energy"], model='additive')

fig = make_subplots(rows=4, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1)

fig.add_trace(go.Scatter(x=date_axis, y=result.observed),
              row=1, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.trend),
              row=2, col=1)

y = result.trend.dropna().values
x = np.arange(len(y))

X, Y = x.reshape(-1,1), y.reshape(-1,1)
fig.add_trace(go.Scatter(x=date_axis[6:], y=LinearRegression().fit(X, Y).predict(X).flatten()),
              row=2, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.seasonal),
              row=3, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.resid),
              row=4, col=1)

fig.update_xaxes(title_text="Time", row=4, col=1)
fig.update_yaxes(title_text="Observed", row=1, col=1)
fig.update_yaxes(title_text="Trend", row=2, col=1)
fig.update_yaxes(title_text="Seasonal", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

fig.update_layout(
    height=600, 
    width=800,
    title_text="Seasonal decompose monthly daily_generated_energy", 
    template="plotly_white",
    showlegend=False
)
fig.show()

In [41]:
#@title
helper = solar.set_index("date").groupby(pd.Grouper(freq='M')).mean()
date_axis = helper.index

result = seasonal_decompose(helper["Gas_mxm"], model='additive')

fig = make_subplots(rows=4, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1)

fig.add_trace(go.Scatter(x=date_axis, y=result.observed),
              row=1, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.trend),
              row=2, col=1)

y = result.trend.dropna().values
x = np.arange(len(y))

X, Y = x.reshape(-1,1), y.reshape(-1,1)
fig.add_trace(go.Scatter(x=date_axis[6:], y=LinearRegression().fit(X, Y).predict(X).flatten()),
              row=2, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.seasonal),
              row=3, col=1)

fig.add_trace(go.Scatter(x=date_axis, y=result.resid),
              row=4, col=1)

fig.update_xaxes(title_text="Time", row=4, col=1)
fig.update_yaxes(title_text="Observed", row=1, col=1)
fig.update_yaxes(title_text="Trend", row=2, col=1)
fig.update_yaxes(title_text="Seasonal", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

fig.update_layout(
    height=600, 
    width=800,
    title_text="Seasonal decompose monthly gas mxm", 
    template="plotly_white",
    showlegend=False
)
fig.show()

In [42]:
#@title
title = "Histogram of daily solar energy production colored by month"
fig = px.histogram(solar, x="daily_generated_energy", nbins = 30, 
                   color="month", 
                   template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [43]:
#@title
title = "Sum of solar energy production in kWh by year"
fig = px.bar(solar.groupby("year", as_index=False).sum(), 
             x="year", y="daily_generated_energy", 
             template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [44]:
#@title
title = "Sum of gas consumption in m3 by year"
fig = px.bar(solar.groupby("year", as_index=False).sum(), 
             x="year", y="Gas_mxm", 
             template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [45]:
#@title
title = "Violin plot of solar energy production by year"

fig = px.violin(solar, x="year", y="daily_generated_energy", 
                template="plotly_white", box=True, points=False, 
                title=title)

fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [46]:
#@title
title = "Violin plot of solar energy production by month"

fig = px.violin(solar, x="month", y="daily_generated_energy", 
                template="plotly_white", title=title, 
                box=True, points="all")
# fig = px.strip(solar, x="month", y="daily_generated_energy", 
#                 template="plotly_white", title=title, 
#                 )

fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [47]:
#@title
title = "Summary of daily solar energy production, total energy consumption and gas consumption by month"

fig = go.Figure()
fig.add_bar(x=solar.groupby("month", as_index=False).sum()["month"], y=solar.groupby("month", as_index=False).mean()["daily_generated_energy"], name="daily_generated_energy")
fig.add_bar(x=solar.groupby("month", as_index=False).sum()["month"], y=solar.groupby("month", as_index=False).mean()["Gas_mxm"], name="Gas_mxm")
fig.add_bar(x=solar.groupby("month", as_index=False).sum()["month"], y=solar.groupby("month", as_index=False).mean()["grid_usage"], name="grid_usage")

# fig.update_layout(barmode="relative")
fig.update_layout(template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [48]:
#@title
title = "Monthly solar energy production over the years"

fig = px.histogram(solar,
                   x="date", y="daily_generated_energy",
                   histfunc="sum",
                   template="plotly_white", title=title)
fig.update_traces(xbins_size="M1")
fig.update_xaxes(showgrid=True, dtick="M2", tickformat="%b\n%Y")
fig.update_layout(bargap=0.1)
# fig.add_trace(go.Scatter(mode="markers", x=solar["date"], y=solar["daily_generated_energy"], name="daily"))
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [49]:
#@title
title = "Daily solar energy production over the years"
fig = px.area(solar,
              x="date", y="daily_generated_energy",
              template="plotly_white", title=title)
fig.update_xaxes(
    dtick="M2",
    tickformat="%b\n%Y")
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [50]:
#@title
title = "Daily grid energy consumption over the years"
fig = px.line(solar,
              x="date", y="grid_usage",
              template="plotly_white", title=title)
fig.update_xaxes(
    dtick="M2",
    tickformat="%b\n%Y")
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [51]:
#@title
title = "Histogram of daily gas consumption colored by months"
fig = px.histogram(solar,
                   x="Gas_mxm", nbins = 30,
                   color="month", template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [52]:
#@title
title = "Daily solar energy production over the years \
colored red at low gas consumption "
fig = px.scatter(solar,
                 x="date", y="daily_generated_energy", 
                 color="is_hot",
                 trendline="lowess",
                 template="plotly_white", title=title,
                 opacity = 0.5,
                 color_continuous_scale=px.colors.sequential.Bluered,)
fig.update_xaxes(
    tickformat="%b\n%Y",
    dtick="M2",
    tickson="labels",)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [53]:
#@title
title = "Daily house energy consumption over the years\
 colored red at low gas consumption "

fig = px.scatter(solar,
                 x="date", y="consumption",
                 color="is_hot",
                 trendline="lowess",
                 template="plotly_white", title=title,
                 opacity = 0.5,
                 color_continuous_scale=px.colors.sequential.Bluered,
                 )
fig.update_xaxes(
    dtick="M2",
    tickformat="%b\n%Y")
fig.show()

fig.write_html(title.replace(" ", "_") + ".html")


In [54]:
#@title
title = "Polar chart of daily solar energy production \
colored red at low gas consumption"
solar["date_to_360"] = (solar["month"]-1) * 30 + solar["day"] - 1


fig = px.scatter_polar(solar,
                       r="daily_generated_energy", 
                       theta="date_to_360", color="is_hot",
                       template="plotly_white", title=title,
                       opacity=0.5,
                       color_continuous_scale=px.colors.sequential.Bluered
                       )

fig.update_layout(
    polar = dict(
      angularaxis = dict(
            thetaunit = "degrees",
            dtick = 30,
            rotation=90,
            direction = "clockwise",
            tickmode="array",
            tickvals=list(range(0, 360,30)),
            ticktext=list(range(1, 13,1))            
            )
    ))

fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [55]:
#@title
title = "Polar chart of daily gas consumption"
fig = px.scatter_polar(solar,
                       r="Gas_mxm", theta="date_to_360",
                       template="plotly_white", color="is_hot", 
                       title=title,
                       color_continuous_scale=px.colors.sequential.Bluered)

fig.update_layout(
    polar = dict(
      angularaxis = dict(
            thetaunit = "degrees",
            dtick = 30,
            rotation=90,
            direction = "clockwise",
            tickmode="array",
            tickvals=list(range(0, 360,30)),
            ticktext=list(range(1, 13,1))            
            )
    ))

fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [56]:
# fig = px.scatter_matrix(solar, dimensions=["Gas_mxm", "daily_generated_energy"], template="plotly_white")
# fig.show()

In [57]:
#@title
title = "2D histogram of gas consumption and solar energy production"
fig = go.Figure(go.Histogram2dContour(
        x = solar["Gas_mxm"],
        y = solar["daily_generated_energy"],
        colorscale = 'electric'
))
fig.update_layout(
    template="plotly_white",
    title=title,
    xaxis_title="Gas_mxm",
    yaxis_title="daily_generated_energy",
)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")


In [58]:
# !zip -q -r charts.zip *html
# from google.colab import files
# files.download("charts.zip")